# Installations:

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.7 MB/s eta 0:00:00


# Imports:

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os
import torch
from torch_geometric.data import Data

In [ ]:
import argparse
import time
from tqdm import tqdm
import copy as cp
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.data import DataLoader, DataListLoader
from torch_geometric.nn import DataParallel
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool, GATConv

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

# set the device to run on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# define the text cleaning function
def clean_text(text):
    # remove URLs and Twitter handles
    text = re.sub(r"http\S+|@\S+", "", text)
    # remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text).lower()
    # tokenize the text into words
    words = nltk.word_tokenize(text)
    # remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # join the words back into a string
    cleaned_text = ' '.join(words)
    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Loading the inputs:

In [47]:
data_dirs = ['twitter15', 'twitter16']

label_file_path = 'sentiments.txt' #'label.txt'
source_tweets_file_path = 'source_tweets.txt'
tree_file_dir = 'tree'
root_path = '/content/drive/MyDrive/retweet_cascade'

num_features = 1

propagation_graph_list = []
context_graph_dict = {}
labels = []
label_dict = {}


for data_dir in data_dirs:
    dir_path = os.path.join(root_path, data_dir)

    with open(os.path.join(dir_path, label_file_path)) as f:
      for line in f:
        label = int(line.strip().split(':')[1])
        label_dict[line.strip().split(':')[0]] = label

    for file_name in os.listdir(os.path.join(dir_path, tree_file_dir)):
        if not file_name.endswith('.txt'):
            continue

        tweet_id = file_name[:-4]
        if len(tweet_id) > 18:
          print(tweet_id[:18])
          tweet_id = tweet_id[:18]

        node_dict = {}
        edge_list = []
        index_count = 0
        with open(os.path.join(dir_path, tree_file_dir, file_name)) as f:
            for line in f:
                parent, child = line.strip().split('->')
                parent, child = parent.replace("'", "").strip(), child.replace("'", "").strip()
                parent = parent.replace("ROOT", str(tweet_id))
                parent = int(parent.split(',')[1])
                child = int(child.split(',')[1])

                if parent not in node_dict:
                    node_dict[parent] = index_count
                    index_count += 1

                if child not in node_dict:
                    node_dict[child] = index_count
                    index_count += 1

                edge_list.append([node_dict[parent], node_dict[child]])

        labels.append(label_dict[tweet_id])

        # features = torch.zeros((1, num_features))
        # with open(os.path.join(dir_path, source_tweets_file_path)) as f:
        #     for line in f:
        #         if line.startswith(tweet_id):
        #             cols = line.strip().split('\t')
        #             features[0, 0] = int(cols[1])  # number of followers
        #             features[0, 1] = int(cols[2])  # number of friends
        #             features[0, 2] = float(cols[3])  # ratio of followers and friends
        #             features[0, 3] = int(cols[4])  # number of history tweets
        #             features[0, 4] = int(cols[5])  # registration time (year)
        #             features[0, 5] = int(cols[6])  # whether verified account or not
        #             break
        features = torch.ones((1, num_features))

        edge_index = torch.tensor(edge_list).t().contiguous()
        x = features.repeat(len(edge_list) + 1, 1)
        data = Data(x=x, edge_index=edge_index, y=label, id=tweet_id)

        propagation_graph_list.append(data)

    with open(os.path.join(dir_path, 'source_tweets.txt')) as f:
      for line in f:
        tweet_id, tweet_text = line.strip().split('\t')

        cleaned_tweet = clean_text(tweet_text)

        inputs = tokenizer(cleaned_tweet, return_tensors='pt', truncation=True, padding=True).to(device)
        outputs = model(**inputs).last_hidden_state

        num_nodes = outputs.shape[1]  # number of words in the tweet
        node_features = outputs.squeeze(0).detach().cpu().numpy()  # word embeddings
        edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()  # fully-connected graph
        data = Data(x=torch.from_numpy(node_features).float(), edge_index=edge_index)

        context_graph_dict[tweet_id] = data

688096910217879554
688437279405305856
688110059553861634
688281442162352128
688441130271289350
688147918113538049
688502260779843587
688461902821855232


In [ ]:
from torch.utils.data import Dataset
from torch_geometric.data import Data
from torch.utils.data import DataLoader

context_graph_list = []
data_list = []

for graph in propagation_graph_list:
  cont = context_graph_dict[graph.id]
  context_graph_list.append(cont)

  data_list.append(Data(
      x_prop = graph.x,
      edge_index_prop = graph.edge_index,
      x_cont = cont.x,
      edge_index_cont = cont.edge_index,
      y = graph.y,
      id = int(graph.id)
      ))


class TweetGraphDataset(Dataset):
    def __init__(self, propagation_graphs, context_graph_list):
        self.propagation_graphs = propagation_graphs
        self.context_graphs = context_graph_list
        
    def __len__(self):
        return len(self.propagation_graphs)
        
    def __getitem__(self, index):
        propagation_graph = self.propagation_graphs[index]
        context_graph = self.context_graphs[index]
        return propagation_graph, context_graph

dataset = TweetGraphDataset(propagation_graph_list, context_graph_list)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [80]:
dataloader = DataLoader(data_list, batch_size=32, shuffle=True)
torch.save(dataloader, '/content/drive/MyDrive/retweets_loader2.pth')

In [ ]:
# torch.save(graph_list, 'retweet_cascade_dataset.pt')

# Craeting the Dataset:

In [ ]:
# from torch_geometric.data import Data

# epoch_len = 120
# land_mark_num = 18
# len = df.shape[0]-1
# overlap = 0.5
# graph_list = []
# order = []
# count = 0
# step = 10

# #edge_index, edge_attr = set_edges(epoch_len, land_mark_num)

# for i in range(1, len-epoch_len, step):
#   x = set_nodes(norm_Xs, norm_Ys, CLs, i, land_mark_num, epoch_len)
#   data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
#   graph_list.append(data)
#   count = count + 1
#   order.append((count, i))

# graph_num = count

* Saving the dataset:


In [ ]:
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

# loader = DataLoader(graph_list, batch_size=16)
# torch.save(loader, '/content/drive/MyDrive/retweets_loader.pth')

In [ ]:
# import os.path as osp

# import torch
# from torch_geometric.data import Dataset, download_url

# root = '/content/drive/MyDrive/Test_loader.pth'

# class Dataset1(Dataset):
#     def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
#         super().__init__(root, transform, pre_transform, pre_filter)

#     def len(self):
#         return len(self.processed_file_names)

#     def get(self, idx):
#         data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
#         return data

# Creating the model:

* some more imports:

In [63]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, VGAE
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import global_mean_pool

from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid

from sklearn.metrics import f1_score

In [ ]:
class GCNSentiment(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, concat=True):
      super(GCNSentiment, self).__init__()

      self.num_features = input_dim
      self.num_classes = output_dim
      self.nhid = hidden_dim
      self.concat = concat

      self.conv1 = GATConv(self.num_features, self.nhid * 2)
      self.conv2 = GATConv(self.nhid * 2, self.nhid * 2)

      self.fc1 = Linear(self.nhid * 2, self.nhid)

      if self.concat:
        self.fc0 = Linear(self.num_features, self.nhid)
        self.fc1 = Linear(self.nhid * 2, self.nhid)
      self.fc2 = Linear(self.nhid, self.num_classes)

    def forward(self, data):
      x, edge_index, batch = data.x, data.edge_index, data.batch

      x = F.selu(self.conv1(x, edge_index))
      x = F.selu(self.conv2(x, edge_index))
      x = F.selu(global_mean_pool(x, batch))
      x = F.selu(self.fc1(x))
      x = F.dropout(x, p=0.5, training=self.training)

      if self.concat:
        news = torch.stack([data.x[(data.batch == idx).nonzero().squeeze()[0]] for idx in range(data.num_graphs)])
        news = F.relu(self.fc0(news))
        x = torch.cat([x, news], dim=1)
        x = F.relu(self.fc1(x))

      x = F.log_softmax(self.fc2(x), dim=-1)
      return x

class CategoricalCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(CategoricalCrossEntropyLoss, self).__init__()

    def forward(self, inputs, targets):
        # inputs shape: (batch_size, num_classes)
        # targets shape: (batch_size,)
        
        # Convert targets to one-hot encoding
        targets = torch.eye(inputs.size(1))[targets].to(inputs.device)
        
        # Calculate loss
        loss = -torch.sum(targets * torch.log_softmax(inputs, dim=1), dim=1)
        
        # Average loss over batch
        loss = torch.mean(loss)
        
        return loss

def one_hot_ce_loss(outputs, targets):
    criterion = nn.CrossEntropyLoss()
    _, labels = torch.max(targets, dim=1)
    return criterion(outputs, labels)

In [82]:
class ContextEmbedding(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ContextEmbedding, self).__init__()
        torch.manual_seed(12345)
        self.gat1 = GATConv(input_dim, hidden_dim)
        self.gat2 = GATConv(hidden_dim, hidden_dim)
        self.gat3 = GATConv(hidden_dim, hidden_dim)

        self.conv_mu = GCNConv(hidden_dim, output_dim)
        # self.lin = torch.nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, edge_index, batch):
        # x = batch.x
        # edge_index = batch.edge_index

        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.gat3(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.conv_mu(x)
        x = F.dropout(x, p=0.4, training=self.training)
        z = global_mean_pool(x, batch)

        return x


class PropagationEmbedding(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, concat=True):
      super(PropagationEmbedding, self).__init__()

      self.num_features = input_dim
      self.num_classes = output_dim
      self.nhid = hidden_dim
      self.concat = concat

      self.conv1 = GATConv(self.num_features, self.nhid * 2)
      self.conv2 = GATConv(self.nhid * 2, self.nhid * 2)

      self.fc1 = Linear(self.nhid * 2, self.nhid)

      if self.concat:
        self.fc0 = Linear(self.num_features, self.nhid)
        self.fc1 = Linear(self.nhid * 2, self.nhid)
      self.fc2 = Linear(self.nhid, self.num_classes)

    def forward(self, x, edge_index, batch):
      # x, edge_index, batch = data.x, data.edge_index, data.batch

      x = F.selu(self.conv1(x, edge_index))
      x = F.selu(self.conv2(x, edge_index))
      x = F.selu(global_mean_pool(x, batch))
      x = F.selu(self.fc1(x))
      x = F.dropout(x, p=0.5, training=self.training)

      if self.concat:
        news = torch.stack([data.x[(data.batch == idx).nonzero().squeeze()[0]] for idx in range(data.num_graphs)])
        news = F.relu(self.fc0(news))
        x = torch.cat([x, news], dim=1)
        x = F.relu(self.fc1(x))

      # x = F.log_softmax(self.fc2(x), dim=-1)
      return x

class AGCNSent(torch.nn.Module):
    def __init__(self, output_dim):
      super(AGCNSent, self).__init__()
      self.propagationEmb = PropagationEmbedding(1, 8, output_dim, concat=True)
      self.contextEmb = ContextEmbedding(768, 1000, output_dim)
      self.fc1 = Linear(output_dim * 2, 2*output_dim)
      self.fc2 = Linear(output_dim * 2, 3)

    def forward(self, data):
      x1 = self.propagationEmb(data.x_prop, data.edge_index_prop, data)
      x2 = self.contextEmb(data.x_cont, data.edge_index_cont, data)
      x = torch.cat([x1, x2], dim=-1)
      x = self.fc1(x)
      x = self.fc2(x)
      x = F.log_softmax(x, dim=-1)

      return x

In [75]:
print(dataset.propagation_graphs[10])

Data(x=[120, 1], edge_index=[2, 119], y=1, id='522684367647559681')


# Training the model:

In [85]:
from torch_geometric.data import Batch

def my_collate(batch):
    return Batch.from_data_list(batch)

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = 1
hidden_dim = 16
output_dim = 16
batch_size = 16
epoch_num = 200

# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

dataloader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
# dataloader = DataLoader(data_list, batch_size=batch_size, shuffle=True, collate_fn=my_collate)

model = AGCNSent(output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

criterion = CategoricalCrossEntropyLoss().to(device)

model.train()
for epoch in range(epoch_num):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    # for batch in dataloader:
    for batch in data_list:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch).to(device)

        m = nn.Sigmoid()

        loss = nn.CrossEntropyLoss()(out, batch.y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch.num_graphs
        total_correct += (out.argmax(dim=-1) == batch.y).sum().item()
        total_samples += batch.num_graphs

    avg_loss = total_loss / 2316
    avg_acc = total_correct / total_samples

    print("Epoch ", epoch, " loss: ", avg_loss, " acc: ", avg_acc)
    #     loss.backward()
    #     optimizer.step()
    #     total_loss += loss.item()
    # print("Epoch ", epoch, " loss: ", total_loss / epoch_num) # note!


model.eval()
pred_y = []
true_y = []

for batch in dataloader:
    batch = batch.to(device)
    with torch.no_grad():
        out = model(batch)
        pred = out.argmax(dim=-1)
        pred_y.append(pred.cpu().numpy())
        true_y.append(batch.y.cpu().numpy())

pred_y = np.concatenate(pred_y, axis=0)
true_y = np.concatenate(true_y, axis=0)

f1 = f1_score(true_y, pred_y, average='macro')
print("F1 score: ", f1)

AttributeError: ignored

# Let's watch the model!

In [ ]:

m = nn.Sigmoid()
print(m(out))

tensor([[0.2185, 0.3076, 0.2164],
        [0.1923, 0.3209, 0.2245],
        [0.1990, 0.3016, 0.2423],
        [0.2032, 0.3138, 0.2234],
        [0.2111, 0.2896, 0.2452],
        [0.2056, 0.3045, 0.2328],
        [0.2130, 0.3117, 0.2167],
        [0.2254, 0.2895, 0.2317],
        [0.1954, 0.3081, 0.2378],
        [0.2032, 0.3133, 0.2240],
        [0.2089, 0.3058, 0.2280],
        [0.1972, 0.3051, 0.2397],
        [0.1929, 0.3245, 0.2192],
        [0.2162, 0.2985, 0.2300],
        [0.2143, 0.3109, 0.2164],
        [0.2114, 0.3045, 0.2273]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib

In [ ]:
def get_embeddings(dataset):
    model.eval()
    with torch.no_grad():
        data = dataset[0]
        embeddings = model.conv2(F.selu(model.conv1(data.x, data.edge_index)), data.edge_index)
        return embeddings.cpu().numpy()


dataset = graph_list
last_layer_embeddings = []

for graph in dataset:
  last_layer_embeddings.append(model.conv2(F.selu(model.conv1(data.x, data.edge_index)), data.edge_index))
  
# last_layer_embeddings = get_embeddings(dataset)


tsne = TSNE(n_components=2)
embeddings_2d = tsne.fit_transform(np.array(last_layer_embeddings))

colors = ['red', 'green', 'blue']
labels = labels
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.show()